In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import os
import time

# Set Paths

In [2]:
# Preprocessed CSVs
preprocessed_data_path = os.path.join('..', 'Preprocessed_Data')


# Path to save CSVs after clipping
save_folder = os.path.join('..', 'Clipped_Data')

# Log file
log_path = os.path.join('..', 'Data', 'Log.xlsx')
df_log = pd.read_excel(log_path, index_col='Subject')

**[Ignore] Path Check**

In [4]:
os.listdir(preprocessed_data_path)

['Preprocessed_Data23.csv',
 'Preprocessed_Data11.csv',
 'Preprocessed_Data14.csv',
 'Preprocessed_Data5.csv',
 'Preprocessed_Data22.csv',
 'Preprocessed_Data9.csv',
 'Preprocessed_Data16.csv',
 'Preprocessed_Data12.csv',
 'Preprocessed_Data15.csv',
 'Preprocessed_Data10.csv',
 'Preprocessed_Data4.csv',
 'Preprocessed_Data18.csv',
 'Preprocessed_Data17.csv',
 'Preprocessed_Data7.csv',
 'Preprocessed_Data3.csv',
 'Preprocessed_Data13.csv',
 'Preprocessed_Data6.csv',
 'Preprocessed_Data20.csv',
 'Preprocessed_Data21.csv',
 'Preprocessed_Data8.csv',
 'Preprocessed_Data19.csv',
 'Preprocessed_Data2.csv',
 'Preprocessed_Data1.csv']

# Functions

In [5]:
def teststarttime(sub):
    TS = df_log["Date"][sub]
    year = TS.year
    month = TS.month
    day = TS.day
    
    TS = df_log["TestStartTime(IST)"][sub]
    hour = TS.hour
    minute = TS.minute
    second = TS.second
    
    return datetime.timestamp(datetime(year, month, day, hour, minute, second))

def clip_data(df, starttime, endtime):   
    df = df[(df["Timestamp"] > starttime) & (df["Timestamp"] <= endtime)] 
    
    # Form filling clipping
    
    return df

def labelling_stress_rest(df, tst):
    cntrl1 = (df["Timestamp"] > tst + DURATIONS["Control1"][0]) & (df["Timestamp"] <= tst + DURATIONS["Control1"][1])
    test = (df["Timestamp"] > tst + DURATIONS["Test"][0]) & (df["Timestamp"] <= tst + DURATIONS["Test"][1])
    cntrl2 = (df["Timestamp"] > tst + DURATIONS["Control2"][0]) & (df["Timestamp"] <= tst + DURATIONS["Control2"][1])
    df["Label_B"] = 1 * (cntrl1 | test | cntrl2)
    df["Label_T"] = (1 * (cntrl1 | cntrl2)) + (2 * test)
    return df

# Main

In [6]:
#              Stage         Start       End
DURATIONS = {
               "Training" : [0         , 3*60 + 2  ],
               "Relax1"   : [3*60 + 2  , 8*60 + 32 ],
               "Control1" : [8*60 + 32 , 17*60 + 6 ],
               "Rest1"    : [17*60 + 6 , 18*60 + 5 ],
               "Test"     : [18*60 + 5 , 30*60 + 7 ],
               "Rest2"    : [30*60 + 7 , 31*60 + 37],
               "Control2" : [31*60 + 37, 40*60 + 9 ],
               "Relax2"   : [40*60 + 9 , 45*60 + 40],
}

rest = ["Training", "Relax1", "Rest1", "Rest2", "Relax2"]
stress = ["Control1", "Test", "Control2"]

EEG_SFREQ = 128

CHANNELS = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

SUB = list(df_log.index.values)

# IMF Features List that includes avg inst freq and variance i.e. 140 features
imf_feats = []
ims = ['IM0', 'IM1', 'IM2', 'IM3', 'IMR']
for i in CHANNELS:
    for j in ims:
        imf_feats.append('EEG.' + i + '.' + j + '.Avg_Inst_Freq')
        imf_feats.append('EEG.' + i + '.' + j + '.Variance')
        
# Preprocessed data
data_dic = {}
for filename in os.listdir(preprocessed_data_path):
    sub = int(filename.split('.')[0]) # Preprocessed_1.csv
    data_dic[sub] = preprocessed_data_path + filename

In [7]:
for sub in SUB[:]:
    print("Subject : ", sub)
    
    print('\tLoading Data')
    df = pd.read_csv(data_dic[sub])
    df = df[['Timestamp'] + imf_feats]
    
    print("\tCalculating test start time")
    tst = teststarttime(sub)
    
    if abs(tst - df['Timestamp'].iloc[0]) > 900:
        raise Exception("\nTest time and Epoc Reading are not corresponding\n")
        
    print("\tClipping EEG_IMF data")
    df = clip_data(df, tst, tst + DURATIONS["Relax2"][1]).reset_index(drop = True)
    
    print("\tLabelling Data")
    df = labelling_stress_rest(df, tst)
    
    print("\tStoring Data")
    store_path = save_folder
    df.to_csv(store_path + str(sub) + ".csv")
    print("\tEND\n")

Subject :  1
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  2
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  3
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  4
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  5
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  6
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  7
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  8
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	Labelling Data
	Storing Data
	END

Subject :  9
	Loading Data
	Calculating test start time
	Clipping EEG_IMF data
	